In [1]:
from framework.platform import PlatformManager
from framework.configs import PlatformConfig, EvalConfig
import os


eval_cfg = EvalConfig(
    inception_score=True,
    fid=True,
    kid=True,
    prc=False,
    fid_infinity=False,
    is_infinity=False,
    clean_fid=False,
    clean_kid=False,
    clean_fid_batch_size=32,
    clean_kid_batch_size=32,
)

platform_cfg = PlatformConfig(
    verbose=True,
    cuda=True,
    save_cpu_ram=False,
    compare_real_to_real=True
)

platform_manager = PlatformManager(real_images_path=os.path.join(os.getcwd(), "original_64"),
                                   eval_config=eval_cfg,
                                   platform_config=platform_cfg)

result_dict = platform_manager.calc_metrics()
result_dict.print()
result_dict.write_to_json(file="is_fid_infty.json")

[INFO]: Real images source found, name:CelebA_Original
[INFO]: 4 different generator sources found. Names:
cifar-10
diffusionStyleGAN2
diffusionVAE
guidedDiffusion_IP
[INFO]: Comparison real-to-real (True)
[START]: Calculating Metrics for cifar-10
[INFO]: Start Calculation IS, Source = cifar-10


Creating feature extractor "inception-v3-compat" with features ['logits_unbiased']
Extracting features from input1


KeyboardInterrupt: 

In [2]:
from framework.platform import ResultDict

results = ResultDict({})
results.read_from_json("all.json")
results.print(round_scores=True)

------------------------------------
Generator Name: cifar-10
------------------------------------
Inception Score Mean: 11.189
Inception Score Std: 0.115
Frechet Inception Distance: 204.998
Kernel Inception Distance Mean: 0.186
Kernel Inception Distance Std: 0.003
Precision: 0.085
Recall: 0.001
F1 Score: 0.002
FID Infinity (Approx.): 225.647
IS Infinity (Approx.): 9.475
Clean FID: 211.253
Clean KID: 0.195
LS: 0.558
------------------------------------
Generator Name: diffusionStyleGAN2
------------------------------------
Inception Score Mean: 3.117
Inception Score Std: 0.028
Frechet Inception Distance: 11.367
Kernel Inception Distance Mean: 0.011
Kernel Inception Distance Std: 0.001
Precision: 0.613
Recall: 0.362
F1 Score: 0.455
FID Infinity (Approx.): 8.723
IS Infinity (Approx.): 3.16
Clean FID: 11.949
Clean KID: 0.011
LS: 0.599
------------------------------------
Generator Name: diffusionVAE
------------------------------------
Inception Score Mean: 3.032
Inception Score Std: 0.03

In [ ]:
metrics_dict = torch_fidelity.calculate_metrics(
    input1=platform_manager.helper.real_images_src.get_dataset(), 
    input2=platform_manager.helper.get_generator_src("diffusionStyleGAN2").get_dataset(), 
    cuda=True, 
    isc=False, 
    fid=False, 
    kid=False, 
    prc=True, 
    verbose=True,
    save_cpu_ram=False,
)

Creating feature extractor "vgg16" with features ['fc2_relu']
Extracting features from input1
Processing samples                                                             
Extracting features from input2
Processing samples                                                             
Precision: 0.6168400049209595
Recall: 0.3585200011730194
F-score: 0.4534725186722886


In [ ]:
from framework.util import img_to_64

img_to_64("D:\Dokumente\SoftwareDev\MasterThesis\GIM-evaluation\cifar-10\\train\\train\\",
             "D:\Dokumente\SoftwareDev\MasterThesis\GIM-evaluation\cifar-10\\64res\\")

(50000, 64, 64, 3) size array saved into celeba64_train.npz


In [ ]:
from third_party.FID_IS_infinity.score_infinity import calculate_FID_infinity_path, calculate_IS_infinity_path

fid_infty = calculate_FID_infinity_path(real_path=platform_manager.helper.real_images_src.folder_path, 
                                        fake_path=platform_manager.get_generator_src("guidedDiffusion_IP").folder_path,
                                        batch_size=128)

print(fid_infty)


d:\Users\qrno9\miniconda3\envs\GIM-Eval\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\Users\qrno9\miniconda3\envs\GIM-Eval\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to C:\Users\qrno9/.cache\torch\hub\checkpoints\inception_v3_google-0cc3c7bd.pth
100%|██████████| 104M/104M [00:04<00:00, 27.2MB/s] 
100%|██████████| 391/391 [02:52<00:00,  2.27it/s]


7.062873860723985


In [ ]:
is_infty = calculate_IS_infinity_path(path=platform_manager.get_generator_src("guidedDiffusion_IP").folder_path,
                                      batch_size=128)

print(is_infty)

d:\Users\qrno9\miniconda3\envs\GIM-Eval\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\Users\qrno9\miniconda3\envs\GIM-Eval\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 391/391 [01:11<00:00,  5.47it/s]


2.8650191160150813


In [ ]:
from cleanfid import fid as clean_fid

from framework.platform import PlatformManager
from framework.configs import PlatformConfig, EvalConfig
import os


eval_cfg = EvalConfig(
    inception_score=False,
    fid=False,
    kid=False,
    prc=False,
    fid_infinity=True,
    is_infinity=True,
)

platform_cfg = PlatformConfig(
    verbose=True,
    cuda=True,
    save_cpu_ram=False,
    compare_real_to_real=True
)

platform_manager = PlatformManager(real_images_path=os.path.join(os.getcwd(), "original_64"),
                                   eval_config=eval_cfg,
                                   platform_config=platform_cfg)

print()

# fid = clean_fid.compute_fid(fdir1=platform_manager.helper.real_images_src.folder_path,
#                             fdir2=platform_manager.get_generator_src("guidedDiffusion_IP").folder_path,
#                             num_workers=0,
#                             batch_size=256)

kid = clean_fid.compute_kid(fdir1=platform_manager.helper.real_images_src.folder_path,
                            fdir2=platform_manager.get_generator_src("guidedDiffusion_IP").folder_path,
                            num_workers=0,
                            batch_size=256)

print(kid)

[INFO]: Real images source found, name:CelebA_Original
[INFO]: 4 different generator sources found. Names:
cifar-10
diffusionStyleGAN2
diffusionVAE
guidedDiffusion_IP

compute KID between two folders
Found 202599 images in the folder d:\Dokumente\SoftwareDev\MasterThesis\GIM-evaluation\original_64


KID original_64 : 100%|██████████| 792/792 [14:18<00:00,  1.08s/it]


Found 50000 images in the folder d:\Dokumente\SoftwareDev\MasterThesis\GIM-evaluation\generated_images\guidedDiffusion_IP


KID guidedDiffusion_IP : 100%|██████████| 196/196 [03:33<00:00,  1.09s/it]


0.011523317377377345


In [1]:
from framework.platform import PlatformManager
from framework.configs import PlatformConfig, EvalConfig
import os



eval_cfg = EvalConfig(
    ls=True,
    ls_n_folds=0,
    ls_n_samples=10000,
    ls_plot_distances=False,
)

platform_cfg = PlatformConfig(
    verbose=True,
    cuda=True,
    save_cpu_ram=False,
    compare_real_to_real=True,
    num_worker=16,
)

platform_manager = PlatformManager(real_images_path=os.path.join(os.getcwd(), "original_64"),
                                   eval_config=eval_cfg,
                                   platform_config=platform_cfg)

result_dict = platform_manager.calc_metrics()
result_dict.print()
result_dict.write_to_json(file="ls.json")

[INFO]: Real images source found, name:CelebA64 (Original)
[INFO]: 4 different generator sources found. Names:
cifar-10
diffusionStyleGAN2
diffusionVAE
guidedDiffusion_IP
[INFO]: Comparison real-to-real (True)
[START]: Calculating Metrics for cifar-10
[INFO]: Start Calculation Likeliness Scores (LS), Source = cifar-10
[INFO]: LS finished
[FINISHED]: Calculating Metrics for cifar-10
{'LS': 0.5572416015235531}
[START]: Calculating Metrics for diffusionStyleGAN2
[INFO]: Start Calculation Likeliness Scores (LS), Source = diffusionStyleGAN2
[INFO]: LS finished
[FINISHED]: Calculating Metrics for diffusionStyleGAN2
{'LS': 0.5932701201504269}
[START]: Calculating Metrics for diffusionVAE
[INFO]: Start Calculation Likeliness Scores (LS), Source = diffusionVAE
[INFO]: LS finished
[FINISHED]: Calculating Metrics for diffusionVAE
{'LS': 0.6044717780069088}
[START]: Calculating Metrics for guidedDiffusion_IP
[INFO]: Start Calculation Likeliness Scores (LS), Source = guidedDiffusion_IP
[INFO]: LS f